In [ ]:
import numpy as np

In [ ]:
!pip install tableprint


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Dec 05 21:30:01 2014

@author: spokutta
"""
#################################
# file names/directories
#################################
import os
logging_dir = os.getcwd()

#################################
# other parameters
#################################
autograd = True

######################################
# Weak Separation Oracle Parameters
######################################
nodeLimit = None
accuracyComparison = 1e-12
ggEps = 1e-08  # accuracy measure in early termination for LP solver


#################################
# Oracle Cache information
#################################
useCache = True
previousPoints = {}


###########################################
# Backtracking Line Search Parameters
###########################################
ls_tau = 0.5
ls_eps = 0.01

###############################
# Algorithm Configuration
###############################
run_config = {
        'solution_only': True,
        'verbosity': 'normal',
        'dual_gap_acc': 1e-06,
        'runningTimeLimit': None,
        'use_LPSep_oracle': True,
        'max_lsFW': 30,
        'strict_dropSteps': True,
        'max_stepsSub': 200,
        'max_lsSub': 30,
        'LPsolver_timelimit': 100,
        'K': 1
        }



In [ ]:
"""Create objective function by name.

Currently, the name is the name of a class from the module with almost
the same name.  The module name is obtained from name by converting
all capital letters to lower case and inserting a _ before them.
Exceptions are capital letters following a _, which are left alone.
All leading _ are stripped from the module name.

Examples:

>>> import numpy as np

>>> f = ObjectiveFunctionFactory.create_function(
...        'standard_parabola', np.arange(3))
>>> float(round(f.evaluate(np.array([1, 0, -1])), 5))
11.0

>>> ObjectiveFunctionFactory.create_function(
...     'square_norm_Ax_minus_b', 4, 3, 2)
... # doctest: +ELLIPSIS
<objective_functions.square_norm_Ax_minus_b.square_norm_Ax_minus_b object at ...>
>>> ObjectiveFunctionFactory.create_function(
...     'lasso_function', 5)
... # doctest: +ELLIPSIS
<objective_functions.lasso_function.lasso_function object at ...>
>>> ObjectiveFunctionFactory.create_function(
...     'sdp_matrixCompletion', size=5, density=1/3,
...     rank_ratio=2/5)
... # doctest: +ELLIPSIS
<objective_functions.sdp_matrix_completion.sdp_matrixCompletion object at ...>
>>> ObjectiveFunctionFactory.create_function(
...     'MatrixCompletion', np.arange(6).reshape(3, 2), 0.1)
... # doctest: +ELLIPSIS
<objective_functions.matrix_completion.MatrixCompletion object at ...>
"""
# TODO: Test the created functions.  Unfortunately, most of them is random, so hard to check for specific values.

import importlib
import re


class ObjectiveFunctionFactory(object):

    @staticmethod
    def create_function(name, *args, **kwargs):
        def module_name(match):
            return '_' + match.group().lower()

        module_name = re.sub(r'(?<!_)[A-Z]', module_name, name) \
                        .lstrip('_')
        module = importlib.import_module('objective_functions.'
                                         + module_name)
        obj_function = getattr(module, name)
        return obj_function(*args, **kwargs)


In [ ]:
import abc


class ObjFunction(abc.ABC):
    """Objective function with gradient.

    >>> import numpy as np
    >>> class TestFunction(ObjFunction):
    ...
    ...     def evaluate(self, value):
    ...         return value[0] ** 2 - value[1] ** 3
    ...
    ...     def gradient(self, value):
    ...         return np.array([2 * value[0], -3 * value[1] ** 2])
    >>> t = TestFunction()
    >>> t.evaluate(np.array([1, -1]))
    2
    >>> t.gradient(np.array([1, -1]))
    array([ 2, -3])
    """
    @abc.abstractmethod
    def evaluate(self, value):
        """Return function value at point value."""
        return

    @abc.abstractmethod
    def gradient(self, value):
        """Return gradient at point value.

        >>> import numpy as np
        >>> class TestFunction(ObjFunction):
        ...
        ...     def evaluate(self, value):
        ...         return np.sum(value * value)
        ...
        ...     def gradient(self, value):
        ...         return super().gradient(value)
        >>> np.around(
        ...     TestFunction().gradient(np.arange(6).reshape(2, 3)),
        ...     5)
        array([[  0.,   2.,   4.],
               [  6.,   8.,  10.]])


        Implementation notes:

        scipy.optimize.approx_fprime doesn't support functions with
        multidimensional array argument:

        >>> import numpy as np
        >>> from scipy.optimize import approx_fprime
        >>> def x_x_transpose(x):
        ...     return np.sum(x * x)
        >>> np.around(approx_fprime(np.arange(6),
        ...                         x_x_transpose,
        ...                         np.sqrt(np.finfo(float).eps)),
        ...           5)
        array([  0.,   2.,   4.,   6.,   8.,  10.])
        >>> np.around(approx_fprime(np.arange(6).reshape(2, 3),
        ...                         x_x_transpose,
        ...                         np.sqrt(np.finfo(float).eps)),
        ...           5) #doctest: +IGNORE_EXCEPTION_DETAIL
        Traceback (most recent call last):
          ...
        ValueError: operands could not be broadcast together with shapes (2,3) (2,)

        Correct output should be:

        array([[  0.,   2.,   4.],
               [  6.,   8.,  10.]])
        """
        from scipy.optimize import approx_fprime
        import numpy as np
        shape = value.shape

        def f(x):
            return self.evaluate(x.reshape(shape))

        gradient = approx_fprime(value.reshape(-1),
                                 f,
                                 np.sqrt(np.finfo(float) .eps))
        return gradient.reshape(shape)

    def check_gradient(self, value, accuracy=None):
        """Verify gradient of function.

        Return True if gradient is correct, otherwise the difference
        in two norm.

        Parameters:

        value: the point at which to verify gradient

        accuracy: allowed error in gradient in two norm, due to
                  numerical errors and gradient estimation.

        Examples:

        >>> import numpy as np
        >>> def f(x):
        ...     return np.sum(x ** 3)
        ...
        >>> def f_grad(x):
        ...     return 3 * x ** 2
        ...
        >>> def f_grad_wrong_shape(x):
        ...     # Flattening array is an error
        ...     return f_grad(x).reshape(-1)
        ...
        >>> def f_wrong_grad(x):
        ...     return 2 * x ** 2
        ...
        >>> test = ObjectiveFunction(f, f_grad)
        >>> test_wrong_shape = ObjectiveFunction(f, f_grad_wrong_shape)
        >>> test_wrong_grad = ObjectiveFunction(f, f_wrong_grad)

        >>> test.check_gradient(np.random.randint(10, size=(2, 3)),
        ...                     10 ** -5)
        True
        >>> test.check_gradient(np.array([[2, 0, 1], [-1, 0, -3]]))
        True
        >>> test_wrong_shape.check_gradient(np.array([[2, 1, 1],
        ...                                           [-1, 0, -3]]))
        Traceback (most recent call last):
            ...
        ValueError: Gradient shape is (6,) instead of (2, 3)


        The float() conversion below is a workaround for type
        np.float64 not providing the shortest repersentation.

        >>> float(round(test_wrong_grad.check_gradient(
        ...     np.array([[2, 0, 1], [-1, 0, -3]])), 5))
        9.94987
        """
        from scipy.optimize import check_grad, approx_fprime
        import numpy as np
        shape = value.shape

        def f(x):
            return self.evaluate(x.reshape(shape))

        if accuracy is None:
            estimated_gradient = approx_fprime(value.reshape(-1), f,
                                    np.sqrt(np.finfo(float).eps))
            accuracy = (np.linalg.norm(estimated_gradient)
                        * np.cbrt(np.finfo(float).eps))

        def grad(x):
            gradient = self.gradient(x.reshape(shape))
            if gradient.shape != shape:
                raise ValueError('Gradient shape is %s instead of %s'
                                 % (gradient.shape, shape))
            return gradient.reshape(-1)

        error = check_grad(f, grad, value.reshape(-1))
        return True if error <= accuracy else error


class ObjectiveFunction(ObjFunction):
    """Explicitly given objective function.

    Example:

    >>> import numpy as np
    >>> def f(x):
    ...     return np.sum(x ** 3)
    ...
    >>> def f_grad(x):
    ...     return 3 * x ** 2
    ...
    >>> test = ObjectiveFunction(f)
    >>> test2 = ObjectiveFunction(f, f_grad)

    >>> test.evaluate(np.array([2, 1]))
    9
    >>> test2.evaluate(np.array([2, 1]))
    9

    >>> np.around(test.gradient(np.array([2, 1])), 5)
    array([ 12.,   3.])
    >>> test2.gradient(np.array([2, 1]))
    array([12,  3])

    >>> test.evaluate(np.array([[2, 1], [-1, -3]]))
    -19
    >>> test2.evaluate(np.array([[2, 1], [-1, -3]]))
    -19

    >>> np.around(test.gradient(np.array([[2, 1], [-1, -3]])), 5)
    array([[ 12.,   3.],
           [  3.,  27.]])
    >>> test2.gradient(np.array([[2, 1], [-1, -3]]))
    array([[12,  3],
           [ 3, 27]])
    """
    def __init__(self, function, gradient=None):
        """Wrap a function into an instance of ``ObjFunction``.

        Parameters:

        function: The objective function as a callable.
        gradient: The gradient of ``function`` as a callable.
                  If omitted the gradient will be estimated.
        """
        self.__function = function
        if gradient is not None:
            self.__gradient = gradient
        else:
            self.__gradient = super().gradient

    def evaluate(self, value):
        return self.__function(value)

    def gradient(self, value):
        return self.__gradient(value)


In [ ]:
from math import inf
import os
import numpy as np
import datetime

################################
# line search functions
################################
def ternary_ls(obj_fct, x, direction, accuracy):
    gamma_ub = 1
    gamma_lb = 0
    # initialize
    y = x + direction  # end point
    endpoint_val = obj_fct.evaluate(y)
    val_y = endpoint_val
    val_x = obj_fct.evaluate(x)
    i = 0
    while abs(val_y - val_x) > accuracy:
        zx = x + 1/float(3) * (y - x)
        zy = x + 2/float(3) * (y - x)
        value_zx = obj_fct.evaluate(zx)
        value_zy = obj_fct.evaluate(zy)
        if value_zx < value_zy:
            y = zy
            gamma_ub = gamma_lb + (gamma_ub-gamma_lb) * 2/3
            val_y = value_zy  # update value y because position of y changed
        else:
            x = zx
            gamma_lb = gamma_lb + (gamma_ub-gamma_lb) * 1/3
            val_x = value_zx  # update value x because position of x changed
        i += 1
    return gamma_lb, i


def backtracking_ls_FW(objectiveFunction, x, grad, direction, steps):
    step_size = 1
    grad_direction = np.inner(grad, direction)

    i = 0
    # assert grad_direction <= 0, 'grad_direction is {}'.format(grad_direction)
    if grad_direction == 0:
        return 0, i

    evalu_oldpint = objectiveFunction.evaluate(x)
    evalu_newpoint = objectiveFunction.evaluate(x + step_size * direction)
    while (evalu_newpoint - evalu_oldpint) > ls_eps * step_size * grad_direction:
        if i > steps:
            if evalu_oldpint - evalu_newpoint >= 0:
                return step_size, i
            else:
                return 0, i
        step_size *= ls_tau
        evalu_newpoint = objectiveFunction.evaluate(x + step_size * direction)
        i += 1
    # assert (evalu_oldpint - evalu_newpoint >= 0)
    return step_size, i

# Linearly Convergent Frank-Wolfe with Backtracking Line-Search
def backtracking_ls_delta(objectiveFunction, x, grad, direction, steps, step_size_ub):
    step_size = step_size_ub
    grad_direction = np.inner(grad, direction)
    # print("grad_direction {}".format(grad_direction))

    i = 0
    assert grad_direction <= 0, 'grad_direction is {}'.format(grad_direction)
    if grad_direction == 0:
        return 0, i

    evalu_oldpint = objectiveFunction.evaluate(x)
    evalu_newpoint = objectiveFunction.evaluate(x + step_size * direction)
    while (evalu_newpoint - evalu_oldpint) > ls_eps * step_size * grad_direction:
        if i > steps:
            if evalu_oldpint - evalu_newpoint >= 0:
                return step_size, i
            else:
                return 0, i
        step_size *= ls_tau
        evalu_newpoint = objectiveFunction.evaluate(x + step_size * direction)
        i += 1
    # assert (evalu_oldpint - evalu_newpoint >= 0)
    return step_size, i


def backtracking_ls_on_alpha(alpha_list, objectiveFunction, s_list, step_size_ub, direction, steps,
                             func_val_improve_last, strict_dropSteps = True):
    """
    backtracking line search method from https://people.maths.ox.ac.uk/hauser/hauser_lecture2.pdf
    used on sub-algorithm
    """

    step_size = step_size_ub
    grad_direction = -np.inner(direction, direction)

    x_old = np.dot(np.transpose(s_list), alpha_list)
    x_new = np.dot(np.transpose(s_list), alpha_list + step_size * direction)  # end point
    evalu_oldpint = objectiveFunction.evaluate(x_old)
    evalu_newpoint = objectiveFunction.evaluate(x_new)

    # relax dropping criterion
    if func_val_improve_last != 'N/A':
        if not strict_dropSteps:
            drop_criteria = min(0.5 * func_val_improve_last, ls_eps)
        else:
            drop_criteria = 0
        if evalu_newpoint <= evalu_oldpint + drop_criteria:
            return step_size, 0, 'P'

    # begin line search
    i = 0
    while (evalu_newpoint - evalu_oldpint) > ls_eps*step_size * grad_direction:
        if i > steps and evalu_newpoint - evalu_oldpint >= 0:
            return 0, i, 'PS'
        step_size *= ls_tau
        x_new = np.dot(np.transpose(s_list), alpha_list + step_size * direction)
        evalu_newpoint = objectiveFunction.evaluate(x_new)
        i += 1
    if evalu_newpoint >= evalu_oldpint:
        return 0, i, 'PS'
    return step_size, i, 'P'


################################
# cache functions:
################################
def inSequence(array, sequence):
    """Return True when Numpy array is an element of sequence.

    >>> inSequence(np.array([1,2,3]), [np.array([0,1,2]),
    ...                                np.array([1.0, 2.0, 3.0])])
    True

    >>> inSequence(np.array([1,2,3]), [np.array([0,1,2]),
    ...                                np.array([-2.0, 1.0, 3.0])])
    False
    """
    for i in sequence:
        if np.all(array == i):
            return True
    return False


def removeFromCache(x):
    """Remove point x from cache if there.
    >>> _ignore = reset_cache()
    >>> for i in range(3):
    ...     _ignore = addToCache(np.array([i]))
    >>> removeFromCache(np.array([2]))
    point deleted from cache, current number of points in cache 2
    >>> removeFromCache(np.array([3]))

    >>> removeFromCache(np.array([1]))
    point deleted from cache, current number of points in cache 1
    """
    global previousPoints
    current_cache_length = len(previousPoints)
    key = hash(x.tostring())
    try:
        del previousPoints[key]
    except KeyError:
        pass
    else:
        assert current_cache_length - len(previousPoints) == 1


def addToCache(x, clean=None):
    global previousPoints
    if clean:
        result = dict(previousPoints)
        current_value = np.inner(x, x)
        for key, y in previousPoints.items():
            if np.inner(x, y) > current_value:
                result.pop(key)
        previousPoints = result
    key = hash(x.tostring())
    if key not in previousPoints:
        previousPoints[key] = x


def checkForCache(c, goal):
    """Search for a cached numpy array with small objective value.

    c: objective
    goal: upper bound on the acceptable objective value.

    >>> reset_cache()

    >>> _ignore = addToCache(np.array([1., 0.]))

    >>> _ignore = addToCache(np.array([0., 1.]))

    >>> checkForCache(np.array([1,2]), goal=1)
    array([ 1.,  0.])

    >>> checkForCache(np.array([2,1]), goal=1)
    array([ 0.,  1.])

    >>> checkForCache(np.array([1,3]), goal=.5)

    """
    global previousPoints
    for x in previousPoints.values():
        if np.inner(c, x) <= goal:
            break
    else:
        x = None
    return x


def checkForPairwiseCache(c, c_tilde, goal):
    mi = inf
    x_plus = None
    mi_tilde = inf
    x_minus = None
    global previousPoints
    for x in previousPoints.values():
        if np.inner(c, x) < mi:
            mi = np.inner(c, x)
            x_plus = x
        if np.inner(c_tilde, x) < mi_tilde:
            mi_tilde = np.inner(c_tilde, x)
            x_minus = x
        if mi + mi_tilde <= goal:
            break
    return x_plus, x_minus


def find_closest_cache(c):
    m = inf
    m_x = None
    global previousPoints
    for x in previousPoints.values():
        if np.inner(c, x) < m:
            m_x = x
            m = np.inner(c, x)
    return m_x


def reset_cache():
    global previousPoints
    # reset global statistic variables
    previousPoints = {}


####################################
# for reporting on console
####################################
def console_header(all_headers, run_config):
    # under normal mode
    header = all_headers[:3] + all_headers[4:6] + [all_headers[7]]
    width = np.array([12, 8, 22, 22, 12, 12])
    # it will be: ['Iteration', 'Type', 'Function Value', 'Dual Bound', '#Atoms', 'WTime']
    if run_config['verbosity'] == 'verbose':
        header += [all_headers[3]]  # add 'Primal Improve' to the console output
        width = np.append(width, 22)
    return header, width


def utils_console_info(all_info, run_config):
    # under normal mode
    info = all_info[:3] + all_info[4:6] + [all_info[7]]
    if run_config['verbosity'] == 'verbose':
        info += [all_info[3]]  # add 'Primal Improve' to the console output
    return info



In [ ]:
import numpy as np
import time
import logging
import os
import tableprint as tp
from tabulate import tabulate
import signal

class GracefulKiller:
    kill_now = False

    def __init__(self):
        signal.signal(signal.SIGINT, self.exit_gracefully)
        signal.signal(signal.SIGTERM, self.exit_gracefully)

    def exit_gracefully(self, signum, frame):
        self.kill_now = True


class Algorithm:
    
    def __init__(self, feasible_region, objectiveFunction, run_config):
        # parameter setting
        self.feasible_region = feasible_region
        self.objectiveFunction = objectiveFunction
        self.run_config = run_config

        self.continueRunning = True
        self.enter_sub = True  # if enter the sub algorithm
        self.func_val_all = np.array([])
        self.dual_bound_all = np.array([])
        self.wallClock_all = np.array([])
        self.start_time = None

        # for saving results
        self.info_table = []
        # there will be 10 columns of information in logging file
        self.header = ['Iteration',
                       'Type',
                       'Function Value',
                       'Primal Improve',
                       'Dual Bound',
                       '#Atoms',
                       'Iteration Time',
                       'WTime',
                       'step size',
                       'num_ls'
                       ]
        self.console_headerName, self.console_headerWidth = console_header(self.header, self.run_config)
        self.it = 1  # main iteration counter
        self.x_old = self.feasible_region.solve()  # returning initial vertex via LP call
        if self.run_config['use_LPSep_oracle']:  # compute its initial bound
            c = objectiveFunction.gradient(self.x_old)
            initial_min_x = self.feasible_region.solve(c)
            phi = np.inner(c, (self.x_old - initial_min_x)) / 2
            self.dual_bound_all = np.append(self.dual_bound_all, phi)
        self.s_list = [self.x_old.copy()]  # construct initial s_list
        self.alpha = np.ones(1)  # weight vector

        self.phi_recomputedIt = None

    def update_return_info(self, val, phi):
        self.func_val_all = np.append(self.func_val_all, val)
        self.wallClock_all = np.append(self.wallClock_all, time.process_time() - self.start_time)
        self.dual_bound_all = np.append(self.dual_bound_all, phi)

    def find_index_s(self, s):
        for i in range(len(self.s_list)):
            if np.all(self.s_list[i] == s):
                return i
        return None

    def fwStep_update_alpha_S(self, step_size, index_s, s):
        if step_size == 1:
            self.s_list = [s]  # set the s list to be this atom only
            self.alpha = np.array([step_size])  # the weight for this atom is 1
        else:
            self.alpha = (1 - step_size) * self.alpha  # update weight for all the atoms originally in the s_list
            if index_s is None:  # s is new
                self.s_list.append(s)
                self.alpha = np.append(self.alpha, step_size)
            else:  # old atom
                self.alpha[index_s] += step_size

    def if_continue(self, phi, step_size, iter_type, killer):
        if phi <= self.run_config['dual_gap_acc']:
            logging.info('Exit Code 2: Achieved required dual gap accuracy, save results, and exit BCG algorithm.')
            return False
        # early stop for when reaching time limit
        if self.run_config['runningTimeLimit'] is not None:
            if self.wallClock_all[-1] > self.run_config['runningTimeLimit']:
                logging.info('Exit Code 3: Reaching time limit, save current results, and exit BCG algorithm.')
                return False
        if killer.kill_now:
            logging.error('Exit Code 0: Keyboard Interruption, save current results and exit BCG algorithm.')
            return False
        if step_size == 0 and iter_type != 'FI' and (self.it != self.phi_recomputedIt):
            logging.error('Exit Code 1: No further primal progress, save current results, and exit BCG algorithm')
            logging.error('Recomputing final dual gap.')
            return False
        return True

    def recompute_phi(self):
        est_phi = self.dual_bound_all[-1]  # estimated phi after current iteration
        c = self.objectiveFunction.gradient(self.x_old)
        s = self.feasible_region.solve(c)
        actual_phi = np.inner(-c, s - self.x_old)  # actual phi after current iteration
        phi = min(est_phi, actual_phi)
        self.dual_bound_all[-1] = phi  # rewrite phi value for after current iteration
        self.phi_recomputedIt = self.it
        return phi

    def run_algorithm(self):
        global console_info
        killer = GracefulKiller()
        self.start_time = time.process_time()
        with tp.TableContext(self.console_headerName, width=self.console_headerWidth) as t:
            if self.run_config['verbosity'] == 'quiet':
                logging.info('Above information is deactivated because of quiet running mode')
            while self.continueRunning:
                if self.enter_sub and self.it > 1:  # check whether to do SiGD iterations
                    current_phi = self.dual_bound_all[-1]
                    new_val = self.func_val_all[-1]

                    grad_x = self.objectiveFunction.gradient(self.x_old)
                    grad_alpha = np.dot(self.s_list, grad_x)
                    min_grad_alpha = np.min(grad_alpha)
                    max_grad_alpha = np.max(grad_alpha)

                    k = 1  # iteration counter for sub iteration
                    SIGD_improve = 'N/A'
                    while self.continueRunning and max_grad_alpha - min_grad_alpha >= current_phi/self.run_config['K']:
                        sigd_start_time = time.process_time()  # starting time of this iteration
                        old_val = new_val.copy()  # function value from last iteration

                        move_direction = -grad_alpha + np.sum(grad_alpha) / len(
                            grad_alpha)  # decomposed negative gradient
                        # compute the upper bound of step size, similar to simplex method
                        d_neg_index = np.where(move_direction < 0)[0]
                        tmp_neg = np.array([-self.alpha[j] / move_direction[j] for j in d_neg_index])
                        eta_ub = np.min(tmp_neg)
                        # line search along move direction with upper bound as eta_ub
                        step_size, num_ls, SIGD_iteration_type = \
                            backtracking_ls_on_alpha(self.alpha, self.objectiveFunction,
                                                           self.s_list, eta_ub,
                                                           move_direction, self.run_config['max_lsSub'],
                                                           SIGD_improve, self.run_config['strict_dropSteps'])

                        if SIGD_iteration_type == 'PS' or k > self.run_config['max_stepsSub']:  # exit SIGD
                            self.enter_sub = False  # will remain False until a new vertex is found
                            if k > self.run_config['max_stepsSub']:
                                SIGD_iteration_type = 'PS'
                            sigd_end_time = time.process_time()  # ending time of this iteration
                            self.update_return_info(old_val, current_phi)  # didn't move
                            info = ['{}'.format(self.it), SIGD_iteration_type,
                                    '{}'.format(old_val),  # didn't move, append old val
                                    'N/A',  # here use 'N/A' instead of zero
                                    '{}'.format(current_phi),
                                    '{}'.format(len(self.s_list)),
                                    '{:.4f}'.format(sigd_end_time - sigd_start_time),
                                    '{:.4f}'.format(sigd_end_time - self.start_time),
                                    '{}'.format(step_size),
                                    '{}'.format(num_ls)
                                    ]
                            if not self.run_config['solution_only']:
                                self.info_table.append(info)
                            console_info = utils_console_info(info, self.run_config)
                            if self.run_config['verbosity'] != 'quiet':
                                t(console_info)
                            self.it += 1
                            break

                        # update alpha and check if there is vertex that needs to be dropped
                        self.alpha += step_size * move_direction
                        if step_size == eta_ub:
                            vertex_toBeDropped = np.argmin(tmp_neg)  # index in d_neg_index
                            vertex_toBeDropped_index = d_neg_index[
                                vertex_toBeDropped]  # retrieve index in the original alpha and s
                            removeFromCache(self.s_list[vertex_toBeDropped_index])
                            self.s_list.pop(vertex_toBeDropped_index)
                            self.alpha = np.delete(self.alpha, vertex_toBeDropped_index)
                            # normalize
                            self.alpha *= 1 / np.sum(self.alpha)
                            SIGD_iteration_type += 'D'

                        # calculate new point and function value
                        self.x_old = np.dot(np.transpose(self.s_list), self.alpha)
                        new_val = self.objectiveFunction.evaluate(self.x_old)
                        SIGD_improve = old_val - new_val
                        grad_x = self.objectiveFunction.gradient(self.x_old)
                        grad_alpha = np.dot(self.s_list, grad_x)
                        min_grad_alpha = np.min(grad_alpha)
                        max_grad_alpha = np.max(grad_alpha)
                        sigd_end_time = time.process_time()  # ending time of this iteration
                        self.update_return_info(new_val, current_phi)
                        # save all the information
                        info = ['{}'.format(self.it), SIGD_iteration_type,
                                '{}'.format(new_val),
                                SIGD_improve,  # corresponding to function value improve
                                '{}'.format(current_phi),
                                '{}'.format(len(self.s_list)),
                                '{:.4f}'.format(sigd_end_time - sigd_start_time),
                                '{:.4f}'.format(sigd_end_time - self.start_time),
                                '{}'.format(step_size),
                                '{}'.format(num_ls)
                                ]
                        if not self.run_config['solution_only']:
                            self.info_table.append(info)
                        console_info = utils_console_info(info, self.run_config)
                        if self.run_config['verbosity'] != 'quiet':
                            t(console_info)
                        k += 1
                        self.it += 1  # so we have all iterations counter
                        self.continueRunning = self.if_continue(current_phi, step_size, SIGD_iteration_type, killer)
                if not self.continueRunning:  # break from outside loop
                    break

                fw_start_time = time.process_time()  # starting time of FW iteration
                # after sub, continue on FW or LCG step
                c = self.objectiveFunction.gradient(self.x_old)
                if self.run_config['use_LPSep_oracle']:  # call weak separation oracle
                    s, iter_type = self.feasible_region.weak_sep(c, self.x_old, True, self.dual_bound_all[-1]/float(self.run_config['K']))
                    direction = s - self.x_old
                    if iter_type != 'FIC':  # if a new vertex is found, enable enter-sub
                        self.enter_sub = True
                    if iter_type == 'FN':  # if cannot find a better point
                        direction = s - self.x_old
                        est_phi = np.inner(-c, direction)
                        if est_phi < 0:  # there is case that the LP solver stops before the optimal solution is found
                            phi = self.dual_bound_all[-1]
                        else:
                            phi = min(self.dual_bound_all[-1]/2, est_phi / 2)
                    else:  # found an improving vertex
                        phi = self.dual_bound_all[-1]
                else:  # FW
                    s = self.feasible_region.solve(c)
                    direction = s - self.x_old
                    phi = np.inner(-c, direction)
                    iter_type = 'F'

                index_s = self.find_index_s(s)
                if self.objectiveFunction.evaluate(s) <= self.objectiveFunction.evaluate(self.x_old) and \
                        len(self.s_list) >= 2*len(self.x_old):  # promote sparsity
                    step_size, num_ls = 1, 'N/A'
                else:
                    step_size, num_ls = backtracking_ls_FW(self.objectiveFunction, self.x_old, c, direction,
                                                                 self.run_config['max_lsFW'])
                # update s and weight: fw update
                self.fwStep_update_alpha_S(step_size, index_s, s)
                # update x and function value
                self.x_old += step_size * direction
                new_val = self.objectiveFunction.evaluate(self.x_old)
                if self.it == 1:
                    func_val_improve = 'N/A'
                else:
                    func_val_improve = self.func_val_all[-1] - new_val
                # only for reporting
                if iter_type == 'FN':
                    reporting_phi = self.dual_bound_all[-1]  # dual bound from last iteration
                else:
                    reporting_phi = phi
                self.update_return_info(new_val, phi)  # save dual bound after current iteration
                if step_size == 0 and iter_type != 'FI' and self.phi_recomputedIt is None:
                    reporting_phi = self.recompute_phi()
                info = ['{}'.format(self.it), iter_type,
                        '{}'.format(new_val),
                        '{}'.format(func_val_improve),
                        '{}'.format(reporting_phi),
                        '{}'.format(len(self.s_list)),
                        '{:.4f}'.format(time.process_time() - fw_start_time),
                        '{:.4f}'.format(time.process_time() - self.start_time),
                        '{}'.format(step_size),
                        num_ls
                        ]
                if not self.run_config['solution_only']:
                    self.info_table.append(info)
                console_info = utils_console_info(info, self.run_config)
                if self.run_config['verbosity'] != 'quiet':
                    t(console_info)
                self.continueRunning = self.if_continue(phi, step_size, iter_type, killer)
                self.it = self.it + 1

        if not self.run_config['solution_only']:
            with open(os.path.join(logging_dir, 'log.txt'), "a") as f:
                print(tabulate(self.info_table, headers=self.header, tablefmt='presto'), file=f)

        self.wallClock_all = self.wallClock_all - self.wallClock_all[0]
        if self.run_config['use_LPSep_oracle']:
            self.dual_bound_all = np.delete(self.dual_bound_all, 0)

        return self.x_old, {
            'func_val_all': self.func_val_all,
            'wallClock_all': self.wallClock_all,
            'dual_bound_all': self.dual_bound_all
        }

In [ ]:
# contains oracles (LMO and Weak Separation)
# contains feasible region construction (unit cube, L1Ball, Simple, or from lp file, Birkhoffpolytope, spectrahedron)

import abc
import numbers
import logging
import numpy as np

class Model(abc.ABC):
    """A generic class for an LP problem.

    Attributes:

    dimension: The number of coordinates feasible solutions have.


    Implementation specific attributes:
    These may be missing.

    model: Dynamic data for optimizing over the model.
    """

    def __init__(self, dimension):
        """Initialize a model."""
        assert (isinstance(dimension, numbers.Integral)
                and dimension >= 0)
        super().__init__()
        self.dimension = dimension

    @abc.abstractmethod
    def minimize(self, cc=None):
        """Minimize objective cc."""
        pass

    def augment(self, cc=None, x=None, goal=None):
        # for models without usage of Gurobi: no early termination, return self.minimize(cc)
        # for models with usage of Gurobi: this method will be redefined in LPsolver.py
        """Find a solution smaller than value goal for objective cc.
        An already known solution is x if not None."""
        return self.minimize(cc)

    def solve(self, cc=None):  # Linear Optimization Oracle
        return self.minimize(cc)

    def weak_sep(self, cc=None, x=None, strict=True, extra_margin=0):  # Weak Separation Oracle
        """Find a solution for cc with value smaller than that of x.
            The value should be smaller by at least extra_margin."""

        if cc is not None and x is not None:
            goal = np.inner(cc, x) - extra_margin
            if strict:
                goal -= accuracyComparison
            else:
                goal += accuracyComparison
            if useCache:
                y = checkForCache(cc, goal)
                if y is not None:
                    # logging.info('---> found cached point with <c, y> value {}'.format(np.inner(cc, y)))
                    return y, 'FIC'
        else:
            logging.info('finding first feasible point ...... ')

        s = self.augment(cc, x, goal)

        if True:
            if useCache:
                addToCache(s)

            # logging.info('condition: {}'.format(np.inner(cc, s) - goal))
            if x is None or cc is None or (np.inner(cc, s) < goal):
                # logging.info('found an improving vertex!')
                return s, 'FI'
            else:
                # logging.info('Did not find better s!')
                # logging.info('function value:{}'.format(np.inner(cc, s)))
                # logging.info('old value:{}'.format(np.inner(cc, x)))
                if useCache:  # add this point to cache
                    s = find_closest_cache(cc)
                return s, 'FN'

In [ ]:
# conditional autograd import
import importlib.util
spec = importlib.util.find_spec("autograd")
if spec is None:
    import numpy as np
else:
    import autograd.numpy as np
    from autograd import grad

import logging

#TODO: Logging shouldn't be configured when run as a module, it is the
#task of the main application.
logging.basicConfig(level=logging.INFO,
                    format='%(message)s')


def init_model(model):
    if isinstance(model, str):  # LP file
        from .LPsolver import initModel_fromFile
        feasible_region = initModel_fromFile(model)
    else:  # user-defined model class
        feasible_region = model
    return feasible_region


def BCG(f, f_grad, model, run_config=None):
    # reset global statistic variables
    reset_cache()

    feasible_region = init_model(model)
    dim = feasible_region.dimension

    # autograd objective function is possible
    if f_grad is None:
        spec = importlib.util.find_spec("autograd")
        if spec is None or autograd is False:
            objectiveFunction = ObjectiveFunction(f, f_grad)
        else:
            logging.info("Trying to use autograd to compute gradient...")
            try:
                objectiveFunction = ObjectiveFunction(f, grad(f))
            except:
                logging.info("Autograd failed. Using numerical approximation.")
                objectiveFunction = ObjectiveFunction(f, f_grad)
                pass
    else:
        objectiveFunction = ObjectiveFunction(f, f_grad)

    logging.info('Dimension of feasible region: {}'.format(dim))

    if f_grad is not None:  # check the user provided gradient oracle
        logging.info('Checking validity of function gradient...')
        sample_checkpoint = np.random.rand(dim)*10.0
        check_res = objectiveFunction.check_gradient(sample_checkpoint)
        assert check_res, 'gradient check error is {}'.format(check_res)

    if run_config is None:  # make it global
        run_config = run_config
    # run algorithm
    algorithmRun = BCGAlgorithm(feasible_region, objectiveFunction, run_config)
    optimal_x, result_dict = algorithmRun.run_algorithm()
    # dual_bound = result_dict['dual_bound_all'][-1]
    # primal_val = result_dict['func_val_all'][-1]

    # if not run_config['solution_only']:  # save the model pickle file in current working folder
    #     import pickle
    #     import os
    #     pickle.dump(result_dict, open(os.getcwd() + '/model_result.p', 'wb'))
    # return optimal_x, dual_bound, primal_val
    return optimal_x


In [ ]:
!pip install autograd


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 13.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# import autograd.numpy as np


# from sklearn import datasets
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split

# digits = datasets.load_digits()

# A, y = digits.data, digits.target
# A = StandardScaler().fit_transform(A)

# # classify small against large digits
# y = (y > 4).astype(np.int_)

# A_train, A_test, y_train, y_test = train_test_split(
#     A, y, train_size=1300, test_size=497, random_state=0)


# # define a model class as feasible region
# class Model_l1_ball(Model):
#     def minimize(self, gradient_at_x=None):
#         result = np.zeros(self.dimension)
#         if gradient_at_x is None:
#             result[0] = 1
#         else:
#             i = np.argmax(np.abs(gradient_at_x))
#             result[i] = -1 if gradient_at_x[i] > 0 else 1
#         return result


# l1Ball = Model_l1_ball(A.shape[1])  # initialize the feasible region as a L1 ball


# # you can construct your own configuration dictionary
# config_dictionary = {
#         'solution_only': False,
#         'verbosity': 'verbose',
#         'dual_gap_acc': 1e-06,
#         'runningTimeLimit': 40,
#         'use_LPSep_oracle': True,
#         'max_lsFW': 30,
#         'strict_dropSteps': True,
#         'max_stepsSub': 1000,
#         'max_lsSub': 30,
#         'LPsolver_timelimit': 100,
#         'K': 1
#         }


# scale_parameter = 5
# # define function evaluation oracle
# def f(x):
#     return np.sum([np.log(np.exp(-y_train[i]*np.dot(scale_parameter*A_train[i], x))+1) for i in range(len(A_train))])


# res = BCG(f, None, l1Ball, config_dictionary)
# print('optimal solution {}'.format(res[0]))
# print('dual_bound {}'.format(res[1]))


# def predict(data, label, weight):
#     correct = 0
#     for i in range(len(data)):
#         y_est = 1/(1+np.exp(-np.dot(scale_parameter*data[i], weight)))
#         if y_est > 0.5:
#             y_est = 1
#         else:
#             y_est = 0
#         if y_est == label[i]:
#             correct += 1
#     return correct/len(data)


# acc_train = predict(A_train, y_train, res[0])
# print('accuracy on the training dataset {}'.format(acc_train))

# acc_test = predict(A_test, y_test, res[0])
# print('accuracy on the testing dataset {}'.format(acc_test))



In [ ]:
class BCGAlgorithm:

    def __init__(self, feasible_region, objectiveFunction, run_config):
        # parameter setting
        self.feasible_region = feasible_region
        self.objectiveFunction = objectiveFunction
        self.run_config = run_config

        self.continueRunning = True
        self.enter_sub = True  # if enter the sub algorithm
        self.func_val_all = np.array([])
        self.dual_bound_all = np.array([])
        self.wallClock_all = np.array([])
        self.start_time = None

        # for saving results
        self.info_table = []
        # there will be 10 columns of information in logging file
        self.header = ['Iteration',
                       'Type',
                       'Function Value',
                       'Primal Improve',
                    #    'Dual Bound',
                       '#Atoms',
                    #    'Iteration Time',
                       'WTime',
                       'step size',
                       'num_ls'
                       ]
        self.console_headerName, self.console_headerWidth = console_header(self.header, self.run_config)
        self.it = 1  # main iteration counter
        self.x_old = self.feasible_region.solve()  # returning initial vertex via LP call
        if self.run_config['use_LPSep_oracle']:  # compute its initial bound
            c = objectiveFunction.gradient(self.x_old)
            initial_min_x = self.feasible_region.solve(c)
            phi = np.inner(c, (self.x_old - initial_min_x)) / 2
            self.dual_bound_all = np.append(self.dual_bound_all, phi)
        self.s_list = [self.x_old.copy()]  # construct initial s_list
        self.alpha = np.ones(1)  # weight vector

        self.phi_recomputedIt = None

    def update_return_info(self, val, phi):
        self.func_val_all = np.append(self.func_val_all, val)
        self.wallClock_all = np.append(self.wallClock_all, time.process_time() - self.start_time)
        self.dual_bound_all = np.append(self.dual_bound_all, phi)

    def find_index_s(self, s):
        for i in range(len(self.s_list)):
            if np.all(self.s_list[i] == s):
                return i
        return None

    def fwStep_update_alpha_S(self, step_size, index_s, s):
        if step_size == 1:
            self.s_list = [s]  # set the s list to be this atom only
            self.alpha = np.array([step_size])  # the weight for this atom is 1
        else:
            self.alpha = (1 - step_size) * self.alpha  # update weight for all the atoms originally in the s_list
            if index_s is None:  # s is new
                self.s_list.append(s)
                self.alpha = np.append(self.alpha, step_size)
            else:  # old atom
                self.alpha[index_s] += step_size

    def if_continue(self, phi, step_size, iter_type, killer):
        if phi <= self.run_config['dual_gap_acc']:
            logging.info('Exit Code 2: Achieved required dual gap accuracy, save results, and exit BCG algorithm.')
            return False
        # early stop for when reaching time limit
        if self.run_config['runningTimeLimit'] is not None:
            if self.wallClock_all[-1] > self.run_config['runningTimeLimit']:
                logging.info('Exit Code 3: Reaching time limit, save current results, and exit BCG algorithm.')
                return False
        if killer.kill_now:
            logging.error('Exit Code 0: Keyboard Interruption, save current results and exit BCG algorithm.')
            return False
        if step_size == 0 and iter_type != 'FI' and (self.it != self.phi_recomputedIt):
            logging.error('Exit Code 1: No further primal progress, save current results, and exit BCG algorithm')
            logging.error('Recomputing final dual gap.')
            return False
        return True

    def recompute_phi(self):
        est_phi = self.dual_bound_all[-1]  # estimated phi after current iteration
        c = self.objectiveFunction.gradient(self.x_old)
        s = self.feasible_region.solve(c)
        actual_phi = np.inner(-c, s - self.x_old)  # actual phi after current iteration
        phi = min(est_phi, actual_phi)
        self.dual_bound_all[-1] = phi  # rewrite phi value for after current iteration
        self.phi_recomputedIt = self.it
        return phi

    def run_algorithm(self):
        # global console_info
        killer = GracefulKiller()
        self.start_time = time.process_time()

        with tp.TableContext(self.console_headerName, width=self.console_headerWidth) as t:
            if self.run_config['verbosity'] == 'quiet':
                logging.info('Above information is deactivated because of quiet running mode')

            while self.it < 10:

                grad_x = self.objectiveFunction.gradient(self.x_old)
                grad_alpha = np.dot(self.s_list, grad_x)
                
                id_min = np.argmin(grad_alpha)
                id_max = np.argmax(grad_alpha)

                w_t = self.feasible_region.solve(grad_x)
                
                a = self.s_list[id_max]
                s_t = self.s_list[id_min]
                local_PW_gap = np.inner(grad_x, a - s_t)
                FW_gap = np.inner(grad_x, self.x_old - w_t)

                if local_PW_gap > FW_gap:

                    d = a - s_t
                    coeff_star = self.alpha[id_max]

                    # line search along move direction with upper bound as eta_ub
                    step_size, num_ls = backtracking_ls_delta(self.objectiveFunction, self.x_old, grad_x, -d, 10, coeff_star)

                    # update alpha and check if there is vertex that needs to be dropped
                    # print("sum alpha {}".format(np.sum(self.alpha)))
                    SIGD_iteration_type = 'Descent'
                    if step_size == coeff_star:
                        # self.alpha += step_size * d
                        vertex_toBeDropped = a
                        vertex_toBeDropped_index = id_max # retrieve index in the original alpha and s
                        self.s_list.pop(vertex_toBeDropped_index)
                        self.alpha = np.delete(self.alpha, vertex_toBeDropped_index)
                        # normalize
                        self.alpha *= 1 / np.sum(self.alpha)
                        # print("step size 1 {}".format(step_size))
                        SIGD_iteration_type = 'Drop'

                else:
                    d = self.x_old - w_t
                    step_size, num_ls = backtracking_ls_delta(self.objectiveFunction, self.x_old, grad_x, -d, 10, 1)
                    # print("step size 2 {}".format(step_size))
                    # update s and weight: fw update                  
                    self.fwStep_update_alpha_S(step_size, None, w_t)
                    SIGD_iteration_type = 'FW'

                self.x_old -= step_size * d
                self.it = self.it + 1
                sigd_end_time = time.process_time()
                new_val = self.objectiveFunction.evaluate(self.x_old)
                info = ['{}'.format(self.it), SIGD_iteration_type,
                        '{}'.format(new_val),  # didn't move, append old val
                        'N/A',  # here use 'N/A' instead of zero
                        # '{}'.format(current_phi),
                        '{}'.format(len(self.s_list)),
                        # '{:.4f}'.format(sigd_end_time - sigd_start_time),
                        '{:.4f}'.format(sigd_end_time - self.start_time),
                        '{}'.format(step_size),
                        '{}'.format(num_ls)
                        ]
                if not self.run_config['solution_only']:
                    self.info_table.append(info)
                console_info = utils_console_info(info, self.run_config)
                if self.run_config['verbosity'] != 'quiet':
                    t(console_info)

        if not self.run_config['solution_only']:
            with open(os.path.join(logging_dir, 'log.txt'), "a") as f:
                print(tabulate(self.info_table, headers=self.header, tablefmt='presto'), file=f)

        # self.wallClock_all = self.wallClock_all - self.wallClock_all[0]
        # if self.run_config['use_LPSep_oracle']:
        #     self.dual_bound_all = np.delete(self.dual_bound_all, 0)

        return self.x_old, 1

In [ ]:
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import autograd.numpy as np

digits = datasets.load_digits()

A, y = digits.data, digits.target
A = StandardScaler().fit_transform(A)

# classify small against large digits
y = (y > 4).astype(np.int_)

A_train, A_test, y_train, y_test = train_test_split(
    A, y, train_size=1300, test_size=497, random_state=0)


# define a model class as feasible region
class Model_l1_ball(Model):
    def minimize(self, gradient_at_x=None):
        result = np.zeros(self.dimension)
        if gradient_at_x is None:
            result[0] = 1
        else:
            i = np.argmax(np.abs(gradient_at_x))
            result[i] = -1 if gradient_at_x[i] > 0 else 1
        return result


l1Ball = Model_l1_ball(A.shape[1])  # initialize the feasible region as a L1 ball


# you can construct your own configuration dictionary
config_dictionary = {
        'solution_only': False,
        'verbosity': 'verbose',
        'dual_gap_acc': 1e-06,
        'runningTimeLimit': 40,
        'use_LPSep_oracle': False,
        'max_lsFW': 30,
        'strict_dropSteps': True,
        'max_stepsSub': 1000,
        'max_lsSub': 30,
        'LPsolver_timelimit': 100,
        'K': 1
        }


scale_parameter = 5
# define function evaluation oracle
def f(x):
    return np.sum([np.log(np.exp(-y_train[i]*np.dot(scale_parameter*A_train[i], x))+1) for i in range(len(A_train))])


res = BCG(f, None, l1Ball, config_dictionary)
print('optimal solution {}'.format(res))
# print('dual_bound {}'.format(res[1]))


def predict(data, label, weight):
    correct = 0
    for i in range(len(data)):
        y_est = 1/(1+np.exp(-np.dot(scale_parameter*data[i], weight)))
        # print(y_est)
        if y_est > 0.5:
            y_est = 1
        else:
            y_est = 0
        if y_est == label[i]:
            correct += 1
    return correct/len(data)


acc_train = predict(A_train, y_train, res)
print('accuracy on the training dataset {}'.format(acc_train))

acc_test = predict(A_test, y_test, res)
print('accuracy on the testing dataset {}'.format(acc_test))


Trying to use autograd to compute gradient...
Autograd failed. Using numerical approximation.
Dimension of feasible region: 64
╭──────────────┬──────────┬────────────────────────┬────────────────────────┬──────────────┬──────────────┬────────────────────────╮
│    Iteration │     Type │         Function Value │                 #Atoms │        WTime │       num_ls │         Primal Improve │
├──────────────┼──────────┼────────────────────────┼────────────────────────┼──────────────┼──────────────┼────────────────────────┤
│            2 │       FW │       871.120122239295 │                      2 │       0.7293 │            2 │                    N/A │
│            3 │       FW │      822.1863840365975 │                      3 │       1.4502 │            2 │                    N/A │
│            4 │  Descent │      811.0875681777478 │                      3 │       2.1978 │            2 │                    N/A │
│            5 │       FW │      791.2785994352005 │                      4

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=20d1bbe1-9699-47d1-9e58-55fbeb4c72ba' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>